In [ ]:
任务：基于task2_data，建立LSTM模型，生成文本：

1、加载本地文本数据，生成字典
2、数据预处理：将数据转化为符合LSTM模型输入要求的数据，确认数据结构；
3、建立LSTM模型，进行模型训练，计算模型在训练、测试数据集的准确率
4、预测” Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines”的后续字符

模型结构：单层LSTM，30神经元
每次使用前30个字符预测第31个字符

In [4]:
# 文本数据加载
# 打开文本数据可用python自带命令
data = open('task2_data').read()
# 移除换行符
data = data.replace('\n','').replace('\r','') # 将换行符替代为什么都没有
print(data)

Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": an

In [5]:
# 字符字母的去重处理
letters = list(set(data))
print(letters)
features = len(letters)
print(features)

['h', 'v', 'A', '(', 'g', 'u', 't', 'p', ':', '.', 'd', 'k', ',', 'l', 'f', 'a', 'b', 'c', ' ', 'e', 'y', 'i', 'x', 'z', '"', 'm', 'n', 'r', 'L', 'o', ')', 's', 'I']
33


In [6]:
# 建立字典
# 数字字符字典
int_to_char = {a:b for a,b in enumerate(letters)}
print(int_to_char)
# 字符数字字典
char_to_int = {b:a for a,b in enumerate(letters)}
print(char_to_int)

{0: 'h', 1: 'v', 2: 'A', 3: '(', 4: 'g', 5: 'u', 6: 't', 7: 'p', 8: ':', 9: '.', 10: 'd', 11: 'k', 12: ',', 13: 'l', 14: 'f', 15: 'a', 16: 'b', 17: 'c', 18: ' ', 19: 'e', 20: 'y', 21: 'i', 22: 'x', 23: 'z', 24: '"', 25: 'm', 26: 'n', 27: 'r', 28: 'L', 29: 'o', 30: ')', 31: 's', 32: 'I'}
{'h': 0, 'v': 1, 'A': 2, '(': 3, 'g': 4, 'u': 5, 't': 6, 'p': 7, ':': 8, '.': 9, 'd': 10, 'k': 11, ',': 12, 'l': 13, 'f': 14, 'a': 15, 'b': 16, 'c': 17, ' ': 18, 'e': 19, 'y': 20, 'i': 21, 'x': 22, 'z': 23, '"': 24, 'm': 25, 'n': 26, 'r': 27, 'L': 28, 'o': 29, ')': 30, 's': 31, 'I': 32}


In [7]:
# time_step
time_step = 30

In [8]:
import numpy as np
from keras.utils import to_categorical
#滑动窗口提取数据
def extract_data(data, slide):    
    x = []
    y = []    
    for i in range(len(data) - slide):
        x.append([a for a in data[i:i+slide]])
        y.append(data[i+slide])        
    return x,y
#字符到数字的批量转化
def char_to_int_Data(x,y, char_to_int):    
    x_to_int = []
    y_to_int = []
    for i in range(len(x)):
        x_to_int.append([char_to_int[char] for char in x[i]])
        y_to_int.append([char_to_int[char] for char in y[i]])    
    return x_to_int, y_to_int
#实现输入字符文章的批量处理，输入整个字符、滑动窗口大小、转化字典
def data_preprocessing(data, slide, num_letters, char_to_int):    
    char_Data = extract_data(data, slide)
    int_Data = char_to_int_Data(char_Data[0], char_Data[1], char_to_int)
    Input = int_Data[0]
    Output = list(np.array(int_Data[1]).flatten())
    Input_RESHAPED = np.array(Input).reshape(len(Input), slide)
    new = np.random.randint(0,10,size=[Input_RESHAPED.shape[0],Input_RESHAPED.shape[1],num_letters])
    for i in range(Input_RESHAPED.shape[0]):
        for j in range(Input_RESHAPED.shape[1]):
            new[i,j,:] = to_categorical(Input_RESHAPED[i,j],num_classes=num_letters)
    return new, Output

In [11]:
X,y = data_preprocessing(data, time_step, features, char_to_int)
print(X)
print(y)

[[[0 0 1 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 1 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 1 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 1 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 1 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 1 0]]]
[31, 29, 25, 19, 6, 21, 25, 19, 31, 18, 17, 15, 13, 13, 19, 10, 18, 25, 15, 17, 0, 21, 26, 19, 18, 21, 26, 6, 19, 13, 13, 21, 4, 19, 26, 17, 19, 12, 18, 21, 31, 18, 21, 26, 6, 19, 13, 13, 21, 4, 19, 26, 17, 19, 18, 10, 19, 25,

In [15]:
# 数据分离
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=0)
print(X.shape,X_train.shape,X_test.shape)
print(len(y),len(y_train),len(y_test))

(11160, 30, 33) (10044, 30, 33) (1116, 30, 33)
11160 10044 1116


In [17]:
# 输出结果的格式转化
y_train_c = to_categorical(y_train,features)
print(y_train_c)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
# 建立LSTM模型
from keras.models import Sequential
from keras.layers import Dense,LSTM
model = Sequential()
model.add(LSTM(units=30,input_shape=(X_train.shape[1],X_train.shape[2]),activation='relu'))
model.add(Dense(units=features,activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30)                7680      
_________________________________________________________________
dense_3 (Dense)              (None, 33)                1023      
Total params: 8,703
Trainable params: 8,703
Non-trainable params: 0
_________________________________________________________________


In [37]:
# 参数配置
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train_c,batch_size=1000,epochs=50)

Epoch 1/50
10044/10044 [==============================] - 1s 127us/step - loss: 1.6937 - accuracy: 0.5290
Epoch 2/50
10044/10044 [==============================] - 1s 95us/step - loss: 1.0578 - accuracy: 0.6768
Epoch 3/50
10044/10044 [==============================] - 1s 94us/step - loss: 0.8115 - accuracy: 0.7550
Epoch 4/50
10044/10044 [==============================] - 1s 91us/step - loss: 0.7011 - accuracy: 0.7951
Epoch 5/50
10044/10044 [==============================] - 1s 89us/step - loss: 0.6547 - accuracy: 0.8065
Epoch 6/50
10044/10044 [==============================] - 1s 88us/step - loss: 0.6163 - accuracy: 0.8213
Epoch 7/50
10044/10044 [==============================] - 1s 86us/step - loss: 0.5865 - accuracy: 0.8337
Epoch 8/50
10044/10044 [==============================] - 1s 85us/step - loss: 0.5615 - accuracy: 0.8361
Epoch 9/50
10044/10044 [==============================] - 1s 85us/step - loss: 0.5444 - accuracy: 0.8424
Epoch 10/50
10044/10044 [=============================

In [26]:
# 训练数据的预测
y_train_predict = model.predict_classes(X_train)
print (y_train_predict)


[19 19 18 ... 19 26 10]


In [27]:
# 训练结果转化字符
y_train_predict_char = [int_to_char[i] for i in y_train_predict]
print(y_train_predict_char)

['e', 'e', ' ', ' ', 'i', ' ', 'i', 'n', 'c', ' ', 'a', 't', 'a', ' ', 'e', ' ', 'e', 'e', 'o', 't', 'g', ' ', 'l', 'a', 'n', 'A', ' ', 'n', 'm', 'i', 'a', ' ', 'l', 'g', 'b', 's', 't', 'e', 'n', 'o', 't', 'i', 'i', ' ', 'a', 'n', 'a', 'e', 'e', ' ', 't', ' ', 'i', 'a', ',', ' ', 'n', 'l', 'e', 'm', 'i', 'i', 'f', 'l', 't', ' ', ' ', 'a', 'a', ' ', ' ', 'e', 't', 'n', 'f', 'b', ' ', 'g', ' ', 'i', 'i', 'c', 'a', ' ', ' ', 'i', 'u', 'g', 'h', 'n', 't', 'e', 'i', 'n', ':', ' ', ' ', 'e', ' ', 'a', ' ', ' ', 'I', 'l', 'c', 'l', 'd', 's', 'n', 'i', 'n', 'v', 'n', 'm', ' ', 'm', 'n', 'c', 'a', 'v', ' ', 'c', 'i', 'a', 'n', ' ', 'a', 'h', 'u', 'A', ' ', ' ', ' ', 'a', 'k', ' ', 'e', ' ', 't', 'h', 'e', 'c', 'n', 'd', 'k', ' ', 'a', 'd', 't', 'i', 't', 'u', ' ', 'n', 't', 't', ' ', ' ', 'n', 'i', 'i', 't', 'k', 'l', 'a', 'I', 'e', 'e', 'e', 'a', 'n', ' ', 'i', 'n', 'n', ' ', 'e', 'm', 'a', 'i', 't', ' ', ' ', 'a', 'm', 'c', ' ', 't', 'i', 'm', 'l', 't', 'i', ' ', ' ', ' ', 'a', ' ', 'd', ' ',

In [28]:
# 准确率
from sklearn.metrics import accuracy_score
accuracy_train = accuracy_score(y_train,y_train_predict)
print(accuracy_train)

0.7190362405416169


In [29]:
y_test_predict = model.predict_classes(X_test)
print (y_test_predict)
accuracy_test = accuracy_score(y_test,y_test_predict)
print(accuracy_test)

[15 21 15 ... 13 18 15]
0.6881720430107527


In [38]:
# 新字符串预测
new_letters = 'Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines'
X_n,y_n = data_preprocessing(new_letters, time_step, features, char_to_int)
y_n_c = to_categorical(y_n,features)
y_n_predict = model.predict_classes(X_n)
y_n_predict_char = [int_to_char[i] for i in y_n_predict]
print(y_n_predict_char)
accuracy_n = accuracy_score(y_n,y_n_predict)
print(accuracy_n)

['s', 'o', 'm', 'e', 't', 'i', 'm', 'e', 's', ' ', 'a', 'a', 'l', 'l', 'd', 'd', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', ' ', 'i', 'n', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 'c', 'e', ',', ' ', 'i', 's', ' ', 'i', 'n', 't', 'e', 'l', 'l', 'i', 'g', 'e', 'n', 'c', 'e', ' ', 'd', 'e', 'm', 'o', 'n', 's', 't', 'r', 'a', 't', 'e', 'd', ' ', 'b', 'y', ' ', 'm', 'a', 'c', 'h', 'i', 'n', 'e', 's']
0.9746835443037974
